## 1.LLMChain的使用


In [ ]:
# from langchain.chains.llm import LLMChain
# from langchain_core.output_parsers import StrOutputParser
# from langchain_openai import ChatOpenAI
# from langchain.prompts import PromptTemplate
# import os
# import dotenv
# dotenv.load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
# chat_model = ChatOpenAI(
# model = "gpt-4o-mini"
# )
# prompt_template = PromptTemplate.from_template(
# template = "桌上有{number}个苹果，四个桃子和 3 本书，一共有几个水果?"
# )
# chain = LLMChain(llm=chat_model,prompt=prompt_template)
# result = chain.invoke(input={"number":"4"})
# print(result)
# print(type(result))

举例2:使用ChatPromptTemplate、verbose的演示

In [45]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_openai import ChatOpenAI
# import os
# import dotenv
#
# dotenv.load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
#
# chat_model = ChatOpenAI(model="gpt-4o-mini")
#
# # 创建聊天提示模板
# prompt_template = ChatPromptTemplate.from_messages([
#     ("system", "你是一个数学高手"),
#     ("human", "帮我解决如下的数学问题：{question}")
# ])
#
# # 使用 LCEL 创建链
# chain = prompt_template | chat_model | StrOutputParser()
#
# # 调用链
# result = chain.invoke({"question": "桌上有4个苹果，四个桃子和3本书，一共有几个水果?"})
# print(result)
# print(type(result))

桌上有4个苹果和4个桃子。要计算水果的总数，我们只需将苹果和桃子的数量相加：

4（苹果） + 4（桃子） = 8（水果）

因此，桌上一共有8个水果。
<class 'str'>


## 3.2 SimpleSequentialChian

In [50]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
# from langchain_openai import ChatOpenAI
# import os
# import dotenv
#
# dotenv.load_dotenv()
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
#
# chat_model = ChatOpenAI(model="gpt-4o-mini")
#
# chainA_template = ChatPromptTemplate.from_messages(
# [
# ("system", "你是一位精通各领域知识的知名教授"),
# ("human", "请你尽可能详细的解释一下：{input}"),
# ]
# )
# chainA_chains = chainA_template | chat_model | StrOutputParser()
# response = chainA_chains.invoke(input={"input":"什么是LangChain？"})
#
# chainB_template = ChatPromptTemplate.from_messages(
# [
# ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
# ("human", "这是针对一个提问的完整的解释说明内容：{description}"),
# ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内"),
# ]
# )
# chainB_chains = chainB_template | chat_model | StrOutputParser()
# # 导入SimpleSequentialChain
# from langchain.chains import SimpleSequentialChain
# # 在chains参数中，按顺序传入LLMChain A 和LLMChain B
# full_chain = SimpleSequentialChain(chains=[chainA_chains, chainB_chains], verbose=True)
# full_chain.invoke({"input":"什么是langChain？"})

RuntimeError: no validator found for <class 'langchain_core.callbacks.base.BaseCallbackHandler'>, see `arbitrary_types_allowed` in Config

举例3：SquentialChain

In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.chains import SequentialChain
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import os
import dotenv

dotenv.load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")

chat_model = ChatOpenAI(model="gpt-4o-mini")

# 第一个链：使用 ChatPromptTemplate 但转换为 LLMChain
chain1_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一位精通各领域知识的知名教授"),
    ("human", "请你尽可能详细的解释一下：{input},并且{action}")
])

# 将 ChatPromptTemplate 转换为字符串格式的 PromptTemplate
chain1_string_template = chain1_prompt.format(input="{input,output}")
chain1_prompt_template = PromptTemplate(
    input_variables=["input","output"],
    template=chain1_string_template
)
chain1 = LLMChain(llm=chat_model, prompt=chain1_prompt_template, output_key="explanation")

# 第二个链：总结
chain2_prompt = ChatPromptTemplate.from_messages([
    ("system", "你非常善于提取文本中的重要信息，并做出简短的总结"),
    ("human", "这是针对一个提问的完整的解释说明内容：{explanation}"),
    ("human", "请你根据上述说明，尽可能简短的输出重要的结论，请控制在20个字以内")
])

chain2_string_template = chain2_prompt.format(explanation="{explanation}")
chain2_prompt_template = PromptTemplate(
    input_variables=["explanation"],
    template=chain2_string_template
)
chain2 = LLMChain(llm=chat_model, prompt=chain2_prompt_template, output_key="summary")

# 创建 SequentialChain
sequential_chain = SequentialChain(
    chains=[chain1, chain2],
    input_variables=["input","output"],
    output_variables=["explanation", "summary"],
    verbose=True
)

# 执行
result = sequential_chain.invoke({"input": "什么是LangChain？"})
print("解释:", result["explanation"])
print("\n总结:", result["summary"])

RuntimeError: no validator found for <class 'langchain_core.callbacks.base.BaseCallbackHandler'>, see `arbitrary_types_allowed` in Config